<a href="https://colab.research.google.com/github/jarvez31/Theta_sequences/blob/main/Main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import librabries

In [ ]:
fol = "/Theta_sequence/"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 KB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/drive


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import skfuzzy as fuzz
import numpy as np
import matplotlib.pyplot as plt
import progressbar
import pickle, random, tqdm, time
from time import sleep
from shapely.geometry import LineString, LinearRing, box, Point, Polygon

#%%
import keras
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, UpSampling2D
import matplotlib.pyplot as plt
from keras import backend as K  
import numpy as np
import math as mt
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D, AveragePooling1D, UpSampling1D
import pickle

from sklearn import preprocessing
from numpy import linalg as LA
from keras.utils.vis_utils import plot_model
from numpy import matlib
import scipy.io
from scipy import signal
from scipy.ndimage import gaussian_filter
import random
from keras import regularizers
from sklearn.metrics import r2_score




# Setup Parameters

In [ ]:
#@title Folder for storage and trajectory name
Train_enc = True #@param {type:"boolean"}
Train_osc = False #@param {type:"boolean"}
traj1 = "traj_1.pk1" #@param {type:"string"}
# test_p =  7#@param {type:"number"}
# img_fol = "traj_four_objs_diffW" #@param {type:"string"}

#@markdown ### Standard deviation for population code for x,y
stdev =  0.1 #@param {type:"slider", min:0, max:1, step:0.01}

#@markdown ### Model Params
pi_use = "no_osc" #@param ["osc", "no_osc"]
act_func = "relu" #@param {type:"string"}
learn_rate = 0.0001 #@param {type:"number"}
mean_freq = 42 #@param {type:"number"}
dt = 0.002 #@param {type:"number"}
beta =  6#@param {type:"number"}

# PI and HD

In [ ]:
#### LOAD TRAJECTORY ######
traj_name = fol + traj1
with open(traj_name, 'rb') as weight:
    dd = pickle.load(weight)
    weight.close()
locals().update(dd)

x=np.asarray(x)
y = np.asarray(y)
vel = np.asarray(vel)
theta = np.asarray(theta)
cyclen = np.asarray(cyclen)

In [ ]:
#%% -----------------------------------Path Integraion------------------------------------##################
n = 100
bf = []
gamma = []
for k in range(n):
    bf.append(2*mt.pi*np.random.normal(mean_freq,stdev)) # base frequency the requnecy is being multiplied with 2pi to conver into angles
    if k < n/2: # couldn't completely understand this if condition, is it encoding direction?
        gamma.append(1)
    else:
        gamma.append(-1)
bf = np.reshape(np.asarray(bf), (n,1))
gamma = np.reshape(np.asarray(gamma), (n,1))

for ii in range(1,len(cyclen)): # Taking from the list of length of velocities
    end = cyclen[ii]
    start = cyclen[ii-1]
    leng = end-start # Duration of a particular velocity vector

    # t = np.linspace(0, dt*leng, num=leng, endpoint=False).reshape((1,leng)) # time corresponding to that length
    # wt = np.matmul(bf,t) # Beta frequency multiplied by time gives number of cycles
    x_temp = x[start:end].reshape((1,leng)) # Gives the position vector
    a = np.matmul(gamma, x_temp) # Direction coded position? Moving away or towards the place field

    a1 = float(beta)*a   # beta looks like parameter coding for theta precision which we will have to tune
    if ii == 1:
        Xarr_l = a1
    else:
        Xarr_l = np.column_stack((Xarr_l, a1))

Xarr = Xarr_l
pi_beta = np.cos(Xarr) # Converting into theta oscilations
print("Shape is: ")
print(pi_beta.shape)
pi_thresh = pi_beta # Theta oscilations sets the thresholds for the place field cells, the crux of theta sequences
mn = (np.matlib.repmat(np.mean(pi_thresh, axis=1), cyclen[ii], 1)) # For repeating the given array
pi_nor = pi_thresh.T - mn # mean centered(normalized) threshold values
PI1d = pi_nor
d = {"gamma":gamma, "PI1d":PI1d, "bf":bf} #, 'obj': small_box}
with open(fol + '/HD_PI.pk1', 'wb') as f:
    pickle.dump(d, f)


Shape is: 
(100, 307775)


# Encoders

In [ ]:
# Train=True
if Train_enc:
    input_data = Input(shape=100,)

    encoder = Dense(30, input_dim = (100,), activation='relu')(input_data)
    decoder = Dense(100, input_dim = (30,), activation='relu')(encoder)

    autoencoder = Model(input_data, decoder)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    autoencoder.compile(optimizer=opt, loss='mse')
    autoencoder.summary()
    plot_model(autoencoder, to_file='phase.png', show_shapes=True, show_layer_names=True)
    encoderModel = Model(input_data, encoder)
    encoderModel.compile(optimizer=opt, loss='mse')
    data = np.array(PI1d[:cyclen[-6],:])
    x_train, x_test, = train_test_split(data, test_size=0.1, random_state=42)

    # Train the model
    autoencoder.fit(x_train, 
                    x_train,
                    epochs= 40,
                    # steps_per_epoch= 100,
                    batch_size= 512,
                    validation_data= (x_test,x_test),
                    validation_steps= 10,
                    shuffle= False)
    encoderModel.save(fol + "encoder_beta9_S1.h5")
    autoencoder.save(fol + "autoencoder_beta9_S1.h5")

#%% 
# ----------------------------------Encoder and Decoder (Model-2)-------------------------------##################
if Train_enc:
    input_data = Input(shape=30,)

    encoder = Dense(30, input_dim = (30,), activation='relu')(input_data)
    decoder = Dense(30, input_dim = (30,), activation='relu')(encoder)
    autoencoder = Model(input_data, decoder)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    autoencoder.compile(optimizer=opt, loss='mse')
    autoencoder.summary()
    plot_model(autoencoder, to_file='phase.png', show_shapes=True, show_layer_names=True)
    encoderModel = Model(input_data, encoder)
    encoder_model1 = tf.keras.models.load_model(fol + "encoder_beta9_S1.h5")
    data1 = encoder_model1.predict(PI1d[:cyclen[-6],:]) 
    x_train1, x_test1, = train_test_split(data1, test_size=0.1, random_state=42)

    # Train the model
    autoencoder.fit(x_train1, 
                    x_train1,
                    epochs= 40,
                    # steps_per_epoch= 100,
                    batch_size= 512,
                    validation_data= (x_test1,x_test1),
                    validation_steps= 10,
                    shuffle= False)
    encoderModel.save(fol + "encoder_beta9_S2.h5")
    autoencoder.save(fol + "autoencoder_beta9_S2.h5")

# Outputs

In [ ]:
#%%-----------------------Loading and plotting the results-------------------------------
#model 1
encoder_model = tf.keras.models.load_model(fol + "encoder_beta9_S1.h5")
encoded = encoder_model.predict(PI1d)

#model 2
encoder2_model = tf.keras.models.load_model(fol + "encoder_beta9_S2.h5")
encoded2 = encoder2_model.predict(encoded)

In [ ]:
if True:
  from matplotlib.pyplot import close
  outs = [encoded2[cyclen[26]:cyclen[27]]] # output from one back and forth cycle
  lim = [0.8]
  outs_n = ["MEC", "LEC"]
  pos_out = pos

  for k in range(len(outs)):
      resp_neurons = np.transpose(outs[k])
      avg = np.mean(abs(resp_neurons))
      std_dev = np.std(resp_neurons)
      print(avg)
      print(std_dev)
      m = np.amax(resp_neurons)
      print(m)
      num = 0
      for j in range(int(np.divide(len(resp_neurons),30))):
          onm = outs_n[k]
          for i in range(30):
              plt.subplot(5,6,i+1)
              thresh = np.amax(resp_neurons[i+num]) * lim[k]
              firr = np.nonzero(resp_neurons[i+num]>thresh)
              firposgrid = pos_out[firr[0], :]
              plt.scatter(firposgrid[:,0], firposgrid[:,1], s = 1, color = 'red', marker='o', zorder = 5)
              plt.plot(env[0], env[1])
              plt.suptitle('output mesh without obj '+str(num)+ ' to '+str(num+30)+' neurons from '+ onm +' layer, threshold=' +str(lim)+" act_func=" + "relu", fontsize = 20, va = 'bottom', ha = 'center')

          figure = plt.gcf() # get current figure
          figure.set_size_inches(16, 10)
          plt.show()
          close()
          num = num + 30